### Model deployment
In this notebook, we will be converting the trained detectron2 model using torchscript and deploying the model on a inference server
#### Steps required
1. loading model
2. run torchscript
3. export model
4. deployment on torchscript
5. inference using rest/gRPC
6. Try repeating with triton inference server

## Reference
https://github.com/facebookresearch/detectron2/tree/main/tools/deploy

In [ ]:
!git clone https://github.com/facebookresearch/detectron2.git

In [ ]:
!pip install ./detectron2

## Importing required libraries

In [ ]:
## python libraries
import os
from typing import Dict ,List, Tuple

## torch libraries
import torch
from torch import Tensor, nn


## detectron libraries
from detectron2.config import get_cfg
from detectron2.modeling import build_model, GeneralizedRCNN, RetinaNet
from detectron2.model_zoo import get_config_file
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader, detection_utils
from detectron2.utils.visualizer import Visualizer
import detectron2.data.transforms as T
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset, print_csv_format
from detectron2.modeling.postprocessing import detector_postprocess
from detectron2.structures import Boxes
from detectron2.utils.env import TORCH_VERSION
from detectron2.utils.file_io import PathManager
from detectron2.utils.logger import setup_logger
from detectron2.projects.point_rend import add_pointrend_config
from detectron2.export import TracingAdapter, dump_torchscript_IR, scripting_with_instances
from detectron2.utils.logger import setup_logger


# image lib
import cv2
import matplotlib.pyplot as plt




In [ ]:
class CONFIG:
    model_zoo_model = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
    pretrained_model_weights = "../input/satorius-train-detectron2-models/model_0007999.pth"
    
    # roi heads
    num_classes = 3
    roi_head_batch_size_per_image =128
    
    #dataset
    val_data = "../input/satorius-segmentation-coco-json/val_coco.json"
    
    #export params
    output_fp = "output"
    export_format = 'torchscript'

In [ ]:
# register train_data
# register_coco_instances(CONFIG.train_dataset_name, {}, CONFIG.train_coco_json, CONFIG.image_root_dir)
# register val data
# register_coco_instances(CONFIG.val_dataset_name, {}, CONFIG.val_coco_json, CONFIG.image_root_dir)

In [ ]:
## setup config
cfg = get_cfg()
cfg.merge_from_file(get_config_file(CONFIG.model_zoo_model))
cfg.MODEL.DEVICE='cpu' 
cfg.MODEL.WEIGHTS = CONFIG.pretrained_model_weights
cfg.MODEL.ROI_HEADS.NUM_CLASSES = CONFIG.num_classes
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = CONFIG.roi_head_batch_size_per_image
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .5

# dataset config
cfg.INPUT.MASK_FORMAT = 'bitmask'
# cfg.DATASETS.TRAIN = (CONFIG.train_dataset_name,)
# cfg.DATASETS.TEST = (CONFIG.val_dataset_name,)
cfg.DATALOADER.NUM_WORKERS = 0

#test config
cfg.TEST.DETECTIONS_PER_IMAGE = 1000

#pointrend
add_pointrend_config(cfg)

cfg.freeze()

In [ ]:
## build model and load in weights
model = build_model(cfg)
DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)
model.to("cpu") # making sure its a cpu model
model.eval();

In [ ]:
# MetadataCatalog.get("../input/satorius-segmentation-coco-json/val_coco.json")

img = cv2.imread("../input/sartorius-cell-instance-segmentation/train/0140b3c8f445.png")
height, width = img.shape[:2]
aug = T.ResizeShortestEdge([cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST], cfg.INPUT.MAX_SIZE_TEST)
image = aug.get_transform(img).apply_image(img)
image = torch.as_tensor(image.astype("float32").transpose(2,0,1)) #CHW as per pytorch
sample_inputs = [{"image": image, "height": height, "width": width}]

## Model forward pass

In [ ]:
with torch.no_grad():
    model_output = model(sample_inputs)

In [ ]:
v = Visualizer(img[:,:,::-1], MetadataCatalog.get(CONFIG.val_data), scale=1.2)
out = v.draw_instance_predictions(model_output[0]['instances'].to('cpu'))
plt.figure(figsize=(20,15))
plt.imshow(out.get_image()[:,:,::-1])

## After we have verify the model, we can export the model

In [ ]:
def export_tracing(torch_model, inputs):
    assert TORCH_VERSION >= (1, 8)
    image = inputs[0]["image"]
    inputs = [{"image": image}]  # remove other unused keys

    if isinstance(torch_model, GeneralizedRCNN):

        def inference(model, inputs):
            # use do_postprocess=False so it returns ROI mask
            inst = model.inference(inputs, do_postprocess=False)[0]
            return [{"instances": inst}]

    else:
        inference = None  # assume that we just call the model directly

    traceable_model = TracingAdapter(torch_model, inputs, inference)

    if CONFIG.export_format == "torchscript":
        ts_model = torch.jit.trace(traceable_model, (image,))
        with PathManager.open("./output/model_cpu.pt", "wb") as f:
            torch.jit.save(ts_model, f)
        dump_torchscript_IR(ts_model, CONFIG.output_fp)
    elif CONFIG.export_format == "onnx":
        with PathManager.open(os.path.join(CONFIG.output_fp, "model.onnx"), "wb") as f:
            torch.onnx.export(traceable_model, (image,), f, opset_version=11)
    logger.info("Inputs schema: " + str(traceable_model.inputs_schema))
    logger.info("Outputs schema: " + str(traceable_model.outputs_schema))

    if CONFIG.export_format != "torchscript":
        return None
    if not isinstance(torch_model, (GeneralizedRCNN, RetinaNet)):
        return None

    def eval_wrapper(inputs):
        """
        The exported model does not contain the final resize step, which is typically
        unused in deployment but needed for evaluation. We add it manually here.
        """
        input = inputs[0]
        instances = traceable_model.outputs_schema(ts_model(input["image"]))[0]["instances"]
        postprocessed = detector_postprocess(instances, input["height"], input["width"])
        return [{"instances": postprocessed}]

    return eval_wrapper

args 
1. format - torchscript
2. export-method - tracing
3. config-file - path to config file
4. sample-image - default None
5. run-eval - True/False
6. output - output directory


In [ ]:
os.makedirs(CONFIG.output_fp, exist_ok=True)
logger = setup_logger()
exported_model = export_tracing(model, sample_inputs)

In [ ]:
!ls /output_cpu

## zipping and exporting

In [ ]:
!zip -r sartorius_torchscript.zip ./output

## Reloading torchscript and testing

Important thing to take note is that image have been resized to [800, 1083]

1. index 0 is the bbox (need to reshape)
2. index 1 is the prediction class
3. index 2 is the mask (Only the raw output, no postprocessing. only the 28x 28 mask is given, the mask have to be reshaped to the bbox size)
4. index 3 is the confidence
5. index 4 is the image size

In [ ]:
import numpy as np
from PIL import Image, ImageDraw
import  matplotlib.pyplot as plt
import random

In [ ]:
model = torch.jit.load("../input/sartorius-torchscripted/model_cpu.pt")

In [ ]:
raw_image = Image.open("../input/sartorius-cell-instance-segmentation/train/0140b3c8f445.png")
pil_image = np.array(raw_image)
rgb_image = np.stack([pil_image,pil_image,pil_image],axis=2)

aug = T.ResizeShortestEdge([cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST], cfg.INPUT.MAX_SIZE_TEST)
augmentation = aug.get_transform(np.stack([rgb_image,rgb_image],axis=-1))
image = augmentation.apply_image(rgb_image)
image = torch.as_tensor(image.astype("float32").transpose(2,0,1)) #CHW as per pytorch

In [ ]:
with torch.no_grad():
    model_output = model(image)

In [ ]:
inverse_augmentation = augmentation.inverse()

In [ ]:
bboxes = inverse_augmentation.apply_box(model_output[0].detach())
masks = model_output[2].detach().numpy()

In [ ]:
rgbimg = Image.new("RGBA",raw_image.size)
rgbimg.paste(raw_image)
original_rgbimg = rgbimg.copy()
original_rgbimg.putalpha(160)
draw = ImageDraw.Draw(rgbimg)

In [ ]:
for bbox, mask in zip(bboxes, masks):
    random_hex = "#"+''.join([random.choice('ABCDEF0123456789') for i in range(6)])
    draw.rectangle(
                    [int(coord) for coord in bbox],
                    width=1,
                    outline=random_hex
                  )
    bbox_width = int(bbox[2]-bbox[0])
    bbox_height = int(bbox[3] - bbox[1])
    
    mask_map = Image.fromarray(np.squeeze(mask), mode='F')
    mask_map = mask_map.resize((bbox_width, bbox_height), resample=Image.BILINEAR)
    mask_map_array = np.array(mask_map) > 0.5
    mask_map = Image.fromarray(mask_map_array)

    draw.bitmap((int(bbox[0]), int(bbox[1])), mask_map, fill=random_hex)
    
    
rgbimg.putalpha(128)
combined_image = Image.alpha_composite(original_rgbimg, rgbimg)

In [ ]:
raw_image

In [ ]:
combined_image

## Encoding to base64

In [ ]:
import io

In [ ]:
buffer = io.BytesIO()
combined_image.save(buffer, format='png')

In [ ]:
buffer.getvalue()

In [ ]:
buffer.getbuffer()